In [1]:
from math import sqrt
import pandas as pd
import numpy as np

from pathlib import Path
from collections import defaultdict
import pickle
import gc

from tqdm import tqdm
from pandarallel import pandarallel

from scipy.sparse import csr_matrix, coo_matrix
import implicit
import catboost

import sys
sys.path.append("..")
from src.utils import *
from src.dataset import *
from src.trending import *

from sklearn.metrics import auc, roc_auc_score, roc_curve
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from catboost import Pool, CatBoostClassifier, cv
from catboost.utils import get_roc_curve, create_cd
from catboost.eval.catboost_evaluation import CatboostEvaluation

pd.set_option('display.max_colwidth', 255)
tqdm.pandas()
pandarallel.initialize(progress_bar=True, nb_workers=8, use_memory_fs=False)

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [3]:
articles = pd.read_csv("../input/articles.csv", index_col=False, dtype={"article_id": str})

In [4]:
import pymorphy2
import nltk
import pickle

In [5]:
articles["detail_desc"]

0                                                                                                                                                                                                                  Jersey top with narrow shoulder straps.
1                                                                                                                                                                                                                  Jersey top with narrow shoulder straps.
2                                                                                                                                                                                                                  Jersey top with narrow shoulder straps.
3         Microfibre T-shirt bra with underwired, moulded, lightly padded cups that shape the bust and provide good support. Narrow adjustable shoulder straps and a narrow hook-and-eye fastening at the back. Without visible seams for greater comfo

In [8]:
from nltk.corpus import stopwords

tokenizer = nltk.RegexpTokenizer(r'\w+')
lemmatizer = nltk.WordNetLemmatizer()
stop_list = stopwords.words("english")


def process(line):
    tokenized_list = tokenizer.tokenize(str(line).lower())
    word_list = []
    for word in tokenized_list:
        word_lemm = lemmatizer.lemmatize(word)
        if word_lemm not in stop_list:
            word_list.append(word_lemm)
    return " ".join(word_list)

In [9]:
articles["desc_norm"] = articles.detail_desc.apply(process)

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA

In [11]:
vectorizer = TfidfVectorizer(min_df=5)
vectors = vectorizer.fit_transform(articles["desc_norm"]).toarray()

In [12]:
pca = PCA(n_components=10)
vectors_pca = pca.fit_transform(vectors)

In [13]:
vectors_pca = ((vectors_pca - vectors_pca.min()) / (vectors_pca.max() - vectors_pca.min()) * 255).astype(np.uint8)

In [14]:
articles["desc_vectors"] = vectors_pca.tolist()

In [15]:
articles[["article_id", "desc_vectors"]].to_csv("../tmp/article_desc_vectors.csv", index=False)